In [2]:
import requests
import datetime

import pandas as pd
import pymongo

from credentials import CONGRESS_API_KEY, PROPUBLICA_API_KEY

get a specific roll call vote:

GET https://api.propublica.org/congress/v1/{congress}/{chamber}/sessions/{session-number}/votes/{roll-call-number}.json

get votes by date:

GET https://api.propublica.org/congress/v1/{chamber}/votes/{year}/{month}.json

get member's vote positions by ID:

GET https://api.propublica.org/congress/v1/members/{member-id}/votes.json

get all members (filter by in-office to get current):

GET https://api.propublica.org/congress/v1/{congress}/{chamber}/members.json


In [ ]:
'''
pull all congress members to get propublica IDs
match with existing entries in congresspeople mongodb using first, last names and chamber = type {'sen', 'rep'}
'''

In [3]:
client = pymongo.MongoClient("mongodb://localhost:27017/") # must be run on olin312-04
db = client["comps"]
votes_collection = db["votes"]
bills_collection = db['bills']
congresspeople_collection = db['congresspeople']

In [22]:
def get_members(congress, chamber):
    url = f"https://api.propublica.org/congress/v1/{congress}/{chamber}/members.json"
    headers = {"X-API-Key": PROPUBLICA_API_KEY}
    response = requests.get(url=url, headers=headers)
    if response.ok:
        return response.json()['results'][0]['members']
    else:
        response.raise_for_status()

In [26]:
'''
unsuitable: only returns 20 most recent votes
'''
def get_vote_positions(member_id):
    url = f"https://api.propublica.org/congress/v1/members/{member_id}/votes.json"
    headers = {"X-API-Key": PROPUBLICA_API_KEY}
    response = requests.get(url=url, headers=headers)
    if response.ok:
        return response.json()#['results'][0]['votes']
    else:
        response.raise_for_status()

In [39]:
all_members

[{'id': 'A000370',
  'title': 'Representative',
  'short_title': 'Rep.',
  'api_uri': 'https://api.propublica.org/congress/v1/members/A000370.json',
  'first_name': 'Alma',
  'middle_name': None,
  'last_name': 'Adams',
  'suffix': None,
  'date_of_birth': '1946-05-27',
  'gender': 'F',
  'party': 'D',
  'leadership_role': None,
  'twitter_account': 'RepAdams',
  'facebook_account': 'CongresswomanAdams',
  'youtube_account': None,
  'govtrack_id': '412607',
  'cspan_id': '76386',
  'votesmart_id': '5935',
  'icpsr_id': '21545',
  'crp_id': 'N00035451',
  'google_entity_id': '/m/02b45d',
  'fec_candidate_id': 'H4NC12100',
  'url': 'https://adams.house.gov',
  'rss_url': 'https://adams.house.gov/rss.xml',
  'contact_form': None,
  'in_office': False,
  'cook_pvi': None,
  'dw_nominate': -0.465,
  'ideal_point': None,
  'seniority': '10',
  'next_election': '2022',
  'total_votes': 997,
  'missed_votes': 5,
  'total_present': 0,
  'last_updated': '2022-12-31 18:30:14 -0500',
  'ocd_id': '

In [42]:
all_members = get_members(117, "house") + get_members(117, "senate")
for member in all_members:
    with_pct = None
    against_pct = None
    try:
        with_pct = member["votes_with_party_pct"]
    except KeyError:
        pass
    try:
        against_pct = member["votes_against_party_pct"]
    except KeyError:
        pass
    congresspeople_collection.update_one(
        {
            "last_name": member['last_name'],
            "first_name": member['first_name'],
            "birthday": member['date_of_birth']
        },
        {"$set": {
            "propublica_id": member['id'],
            'dw_nominate': member['dw_nominate'],
            "votes_with_party_pct": with_pct,
            "votes_against_party_pct": against_pct}})

In [10]:
def get_congress_years(congress_num):
    url = f"https://api.congress.gov/v3/congress/{congress_num}/"
    param_dict = {
        "api_key": CONGRESS_API_KEY,
    }
    response = requests.get(url=url, params=param_dict)
    if response.ok:
        this_congress = response.json()['congress']
        return int(this_congress['startYear']), int(this_congress['endYear'])
    else:
        response.raise_for_status()

In [4]:
def get_vote_uris(year):
    vote_uris = []
    for month in ["01", "02", "03", "04", "05", "06", "07", "08", "09", "10", "11", "12"]:
        url = f"https://api.propublica.org/congress/v1/both/votes/{year}/{month}.json"
        headers = {"X-API-Key": PROPUBLICA_API_KEY}
        response = requests.get(url=url, headers=headers)
        if response.ok:
            try:
                vote_uris.extend([vote['vote_uri'] for vote in response.json()['results']['votes']])
            except KeyError:
                print(f"Missing fields in query for {year}-{month}")
        else:
            response.raise_for_status()
    return vote_uris

In [5]:
vote_uris = get_vote_uris(2021) + get_vote_uris(2022)

In [36]:
def store_votes_by_bill(vote_uri, votes_collection):
    response = requests.get(url=vote_uri, headers={"X-API-Key": PROPUBLICA_API_KEY})
    if response.ok:
        vote_data = response.json()['results']['votes']['vote']
        try:
            bill_id = vote_data['bill']['bill_id']
            store_bill_data(bill_id)
            
        except KeyError:
            bill_id = None
            print(vote_data)    
        
        for position in vote_data['positions']:
            vote_doc = {
                'bill_id': bill_id,
                'member_propublica_id': position['member_id'],
                'vote_position': position['vote_position'],
                'description': vote_data['description'],
                'vote_type': vote_data['vote_type'],
                'date': vote_data['date']
            }
            votes_collection.insert_one(vote_doc)
        
    else:
        response.raise_for_status()

1949

In [8]:
response = requests.get(url=vote_uris[1], headers={"X-API-Key": PROPUBLICA_API_KEY})
vote_data = response.json()['results']['votes']['vote']
bill_uri = vote_data['bill']['api_uri']
bill_response = requests.get(url=bill_uri, headers={"X-API-Key": PROPUBLICA_API_KEY})
bill_response.json()

{'status': 'OK',
 'copyright': 'Copyright (c) 2023 Pro Publica Inc. All Rights Reserved.',
 'results': [{'bill_id': 'sres16-117',
   'bill_slug': 'sres16',
   'congress': '117',
   'bill': 'S.RES.16',
   'bill_type': 'sres',
   'number': 'S.RES.16',
   'bill_uri': 'https://api.propublica.org/congress/v1/117/bills/sres16.json',
   'title': 'A resolution to provide for related procedures concerning the article of impeachment against Donald John Trump, President of the United States.',
   'short_title': 'A resolution to provide for related procedures concerning the article of impeachment against Donald John Trump, President of the United States.',
   'sponsor_title': '',
   'sponsor': 'Charles E. Schumer',
   'sponsor_id': 'S000148',
   'sponsor_uri': 'https://api.propublica.org/congress/v1/members/S000148.json',
   'sponsor_party': 'D',
   'sponsor_state': 'NY',
   'gpo_pdf_uri': None,
   'congressdotgov_url': 'https://www.congress.gov/bill/117th-congress/senate-resolution/16',
   'govtr

In [4]:
congress = 117
url = f"https://api.propublica.org/congress/v1/{congress}/both/bills/introduced.json"
offset = 0
while True:
    response = requests.get(url=url, headers={"X-API-Key": PROPUBLICA_API_KEY}, params={"offset": offset})
    results = response.json()['results']
    bills = results[0]['bills']
    if bills == []:
        break
    offset += 20
    bills_collection.insert_many(bills)


In [6]:
response.json()

{'status': 'OK',
 'copyright': 'Copyright (c) 2023 Pro Publica Inc. All Rights Reserved.',
 'results': [{'congress': 117,
   'chamber': 'Both',
   'num_results': 0,
   'offset': 400020,
   'bills': []}]}